# Regesten in MOM
## Spracherkennung
1. Auslesen der Regestentexte (`cei:abstract`) ohne `cei:foreign` und mit der `atom:id`
    regesten-?.txt (aber mit `cei:foreign`)
2. Einlesen der Regesten:
    Jede Zeile ist ein Regest
    Am Anfang steht in [] die atom:id
2. Erkennen der Sprache z.B. mit python langdetect (Voraussetzung ist die Installation von https://pypi.python.org/pypi/langdetect)
3. Speichern der Sprache im `@xml:lang` Attribut

In [1]:
import re
datei='regesten-6.txt'
regesten= []
längen=[]
with open(datei, 'r', encoding="utf-8") as fh:
    for l in fh:
        text = re.search('\[(tag:www.monasterium.net,2011:/charter/.*?)\] (.*?)$', l)
        if text :
            length = len(text.group(2).split(' '))
            if length > 5 : # Die Bedingung versucht noch ein nGram-Fehler unten abzufangen
                regest =[text.group(1), text.group(2), length]
                längen.append(length)
                regesten.append(regest)
min(längen), max(längen), len(regesten)

(6, 1067, 4868)

In [2]:
# Nun erkenne ich die Sprache (und zwar nur eine pro Regest). Kann ein wenig brauchen
from langdetect import detect
import pandas as pd
regesten_lang = []
for regest in regesten:
    if regest[1]:
        regest.append(detect(regest[1]))
        regesten_lang.append(regest)
dfRegesten = pd.DataFrame(regesten_lang,  columns = ["atom:id", "Regest", "length", "lang"])
dfRegesten.head()

,atom:id,Regest,length,lang
0,"tag:www.monasterium.net,2011:/charter/DE-HStAM...","Hermann von Netra (Netere) [Gem. Ringgau, Werr...",83,de
1,"tag:www.monasterium.net,2011:/charter/AT-StiAL...",Bischof Ovdalricus von Passau weiht auf Bitte ...,191,fr
2,"tag:www.monasterium.net,2011:/charter/AT-StiAL...","Bischof Chunradus von Passau bezeugt, daß Pfar...",181,de
3,"tag:www.monasterium.net,2011:/charter/AT-StiAL...","Wolfgerus de Arnperch bezeugt, daß ihm Weiccar...",101,de
4,"tag:www.monasterium.net,2011:/charter/AT-StiAL...",Die Töchter der Anna der Reinprehtinne Agnes m...,90,de


Ich kann mit detect_langs auch die Wahrscheinlichkeit / Verteilung von mehreren Sprachen errechnen. Das wäre eine Möglichkeit, Texte auszuschließen, die zu wenig klar einer Sprache zugewiesen werden können.

In [3]:
from langdetect import detect_langs #Sprachwahrscheinlichkeit/Sprachmöglichkeiten errechnen
detect_langs(dfRegesten.iloc[1]['Regest']) 

[ca:0.29331705994881085,
 fr:0.2857123768557359,
 it:0.14363918294410338,
 de:0.14285636460803203,
 en:0.13444409840030452]

*Achtung*, das Ergebnis zeigt, daß die Sprachzuweisung in unklaren Fällen nicht stabil ist!

Noch eine ganz andere Möglichkeit zur Spracherkennung wären Webservices wie die Yandex API:
(key: trnsl.1.1.20170616T153230Z.2c48c10e4153c210.f3fb4b9a8437827494a4a4e69b0cf4feb432dbbd )
https://tech.yandex.com/translate/doc/dg/reference/detect-docpage/

Das Problem kurzer Texte könnte mich einholen?

Es gibt folgende Sprachen:

In [3]:
print(set(dfRegesten['lang']))

{'da', 'es', 'sv', 'sk', 'fr', 'cs', 'sl', 'en', 'ca', 'de'}


## NER in den Regesten
mit spacy die NE erkennen
(und in einen eigenen DataFrame schreiben mit : NE, type, atom:id)
Ziel wäre 
1. Testen wie gut die Erkennung ist (z.B. durch Vergleich mit cei:back
3. Markup in die Regesten einfügen oder im cei:back
3. Resolving über Webservices (geonames, gnd/viaf, wikipedia/wikidata)
2. Ergänzen der MOM-Daten mit den Identifikatoren (evtl. auch normalisierten Formen?)

Dazu muß ich
1. spacy laden und alle notwendigen Sprachmodelle laden (python -m spacy download _lang_)
2. testen, ob die Sprachmodelle
2. die Regestenliste iterieren
2. jedes Regest durch spacy.nlp schicken
4. die NE adressieren und auslesen


In [4]:
# Die im Corpus vorkommenden Modelle laden
import os
import spacy
langs_available={'de': 'de_core_news_sm', 'es': 'es_core_news_sm', 'it': 'it_core_news_sm', 'xx': 'xx_ent_wiki_sm'} 
# Da gibt es noch ein paar wenige mehr
langs=set(dfRegesten['lang'])
nlps={}
for i in langs:
    if i in langs_available.keys():
        nlps[i] = spacy.load(langs_available[i])
    else:
        nlps[i] = spacy.load(langs_available['xx'])
print(nlps)

{'da': <spacy.lang.xx.MultiLanguage object at 0x0000018FA49C3198>, 'es': <spacy.lang.es.Spanish object at 0x0000018FB3C5BEB8>, 'sv': <spacy.lang.xx.MultiLanguage object at 0x0000018FB3C642B0>, 'sk': <spacy.lang.xx.MultiLanguage object at 0x0000018FC7D4CC88>, 'fr': <spacy.lang.xx.MultiLanguage object at 0x0000018FDF9D05F8>, 'cs': <spacy.lang.xx.MultiLanguage object at 0x0000018F851B3F28>, 'sl': <spacy.lang.xx.MultiLanguage object at 0x0000018F98A9B8D0>, 'en': <spacy.lang.xx.MultiLanguage object at 0x0000019018DB02B0>, 'ca': <spacy.lang.xx.MultiLanguage object at 0x0000019035EC9BA8>, 'de': <spacy.lang.de.German object at 0x000001906B345F60>}


In [5]:
# Jedem Regest die Named Entities mit spacy zuweisen
# Sprachen aber besser mit dem Treetagger repräsentiert? Zumindest gibt es da sogar Modelle für Latein und Mittelhochdeutsch
NEs=[]
for i in range(len(dfRegesten)):
    regest=dfRegesten.iloc[i]['Regest']
    lang=dfRegesten.iloc[i]['lang']
    atomid=dfRegesten.iloc[i]['atom:id']
    txt_nlp=nlps[lang](regest)
    for entity in txt_nlp.ents:
        #print(entity.text, entity.label_)
        ent = [atomid, lang, entity.text, entity.label_]
        NEs.append(ent)
    #dfRegesten.iloc[i]['NEs']=txt_nlp.ents # Hier will ich nun die Named Entities dem DataFrame hinzufügen, damit ich sie weiterverarbeiten kann

In [13]:
dfNEs = pd.DataFrame(NEs,  columns = ["atom:id", "lang", "NE.text", "NE.label"])
print(len(dfNEs), 'NEs')
print('Distinkt', len(set(dfNEs['NE.text'])))
dfNEs[:10]

38721 NEs
Distinkt 19551


,atom:id,lang,NE.text,NE.label
0,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Martin,PER
1,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Otto,PER
2,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Formbach,LOC
3,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Ernst,PER
4,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Heinrich,PER
5,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Kremsmünster,LOC
6,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Waltse Haubtman,PER
7,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Ens,ORG
8,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Klosters Kremsmünster,LOC
9,"tag:www.monasterium.net,2011:/charter/AT-StiAK...",de,Wandels,PER


## Verifizierung
Das wäre jetzt ein händische Arbeit, oder?

In [14]:
# Erst mal rausschreiben
with open(datei+'-NEs.csv', 'w', encoding='utf-8') as fh:
    dfNEs.to_csv(fh, sep=',', header=True)

## Entity identification
Das wäre jetzt der nächste Schritt, bei dem ich meine NEs mit Quellen aus dem Netz / aus dem übrigen MOM verbinden würde.

Für die Regestenidentifikation könnte das insofern nützlich sein, als es ermöglichen würde, Gemeinsamkeiten in den NEs höher zu gewichten.